In [7]:
file_path = "your path"

with open(file_path, 'r', encoding='utf-8') as file:
    oz_text = file.read()

# Print the first 500 characters to verify the content
print(oz_text[:500])


  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW YORK


  [Illustration]


  COPYRIGHT 1908 BY L. FRANK BAUM

  ALL RIGHTS RESERVED


         *       *       *       *       *


  [Illustration]


  DEDICATED TO HARRIET A. B. NEAL.


         *       *       *       *       *


To My Readers


It's no use; no use at all. The children won't let me


In [8]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow.keras.utils as keras_utils

sentences = oz_text.split('. ')
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)

sequences = tokenizer.texts_to_sequences(sentences)

# Create input sequences and labels
input_sequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        n_gram_sequence = sequence[:i+1]
        input_sequences.append(n_gram_sequence)



In [9]:
# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and labels
train_sequences, labels = input_sequences[:,:-1],input_sequences[:,-1]
vocab_size = len(tokenizer.word_index) + 1  
labels = keras_utils.to_categorical(labels, num_classes=vocab_size)

# Splitting the data into training and testing sets
train_sequences, test_sequences, train_labels, test_labels = train_test_split(train_sequences, labels, test_size=0.2, random_state=42)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

vocab_size = len(tokenizer.word_index) + 1  # Plus 1 for padding token
embed_dim = 64  # Size of the embedding vector

model = Sequential([
    Embedding(vocab_size, embed_dim, input_length=max_sequence_len-1),
    LSTM(64),
    Dense(vocab_size, activation='softmax')
])


In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Adjust this based on your data preparation step
model.fit(train_sequences, train_labels, epochs=20, validation_data=(test_sequences, test_labels))


Epoch 1/20
1040/1040 [==============================] - 64s 61ms/step - loss: 3.5381 - accuracy: 0.2959 - val_loss: 6.6886 - val_accuracy: 0.1299
Epoch 2/20
1040/1040 [==============================] - 63s 61ms/step - loss: 3.3464 - accuracy: 0.3182 - val_loss: 6.8036 - val_accuracy: 0.1278
Epoch 3/20
1040/1040 [==============================] - 62s 60ms/step - loss: 3.2113 - accuracy: 0.3350 - val_loss: 6.8958 - val_accuracy: 0.1270
Epoch 4/20
1040/1040 [==============================] - 62s 60ms/step - loss: 3.0904 - accuracy: 0.3551 - val_loss: 6.9970 - val_accuracy: 0.1294
Epoch 5/20
1040/1040 [==============================] - 63s 61ms/step - loss: 2.9770 - accuracy: 0.3735 - val_loss: 7.0923 - val_accuracy: 0.1221
Epoch 6/20
1040/1040 [==============================] - 62s 60ms/step - loss: 2.8738 - accuracy: 0.3921 - val_loss: 7.1825 - val_accuracy: 0.1229
Epoch 7/20
1040/1040 [==============================] - 66s 63ms/step - loss: 2.7770 - accuracy: 0.4097 - val_loss: 7.2948 -

KeyboardInterrupt: 

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_text(model, tokenizer, start_seed, num_words, max_sequence_len):
    """
    Generate text using a trained model.

    Parameters:
        model: Trained Keras model for text generation.
        tokenizer: Tokenizer used for the model.
        start_seed: Initial seed text to start generation.
        num_words: Number of words to generate.
        max_sequence_len: The maximum length of sequences used by the model.

    Returns:
        A string containing the generated text.
    """
    text_generated = start_seed

    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([text_generated])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        text_generated += " " + output_word

    return text_generated


In [28]:
seed_text = "there is"
next_words = 9  # Number of words to generate

# Generate the text
generated_text = generate_text(model, tokenizer, seed_text, next_words, max_sequence_len)

print(generated_text)


there is no use to be all right answered the boy
